In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import re
from fake_useragent import UserAgent
import numpy as np
from selenium import webdriver
import pickle
from retrying import retry

In [3]:
def getCookies():
    chromePath = '/Users/haizhuhong/chromedriver'
    wd = webdriver.Chrome(executable_path= chromePath)
    url = 'https://account.dianping.com/login?redir=http%3A%2F%2Fwww.dianping.com%2F'
    wd.get(url)
    iframe = wd.find_element_by_xpath('//*[@id="J_login_container"]/div/iframe')
    wd.switch_to_frame(iframe)
    icon_pc = wd.find_element_by_xpath('/html/body/div/div[2]/div[5]/span')
    icon_pc.click()
    time.sleep(2)
    name_login = wd.find_element_by_xpath('//*[@id="tab-account"]')
    name_login.click()
    time.sleep(2)
    
    wd.find_element_by_id('account-textbox').send_keys('**********')
    wd.find_element_by_xpath('//input[@id="password-textbox"]').send_keys('*******')
    wd.find_element_by_xpath('//button[@id="login-button-account"]').click()
    
    cookies=wd.get_cookies()
    pickle.dump(cookies, open('cookies.txt','wb'))
    print(cookies)
    time.sleep(5)
    wd.quit()


In [4]:
def getProxy():
    
    targetUrl = "http://www.baidu.com"
    
    proxy_host = "dyn.horocn.com"
    proxy_port = "50000"

    proxy_username = "xxxxxxx"
    proxy_pwd = "xxxxxxx"
   
    proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
        "host": proxy_host,
        "port": proxy_port,
        "user": proxy_username,
        "pass": proxy_pwd,
    }
    
    proxies = {
        'http': proxyMeta,
        'https': proxyMeta,
    }
    
    return proxies

In [5]:
def getUserAgent():
    user_agent = UserAgent().random
    return user_agent

In [6]:
def getReferer():
    referers=['http://www.dianping.com/chongqing/ch10/g110r1608', 
              'http://www.dianping.com/chongqing/ch10/g110r1621', 
              'http://www.dianping.com/chongqing/ch10/g110r8370', 
              'http://www.dianping.com/chongqing/ch10/g110r1610', 
              'http://www.dianping.com/chongqing/ch10/g110r8371', 
              'http://www.dianping.com/chongqing/ch10/g110r1616', 
              'http://www.dianping.com/chongqing/ch10/g110r1634', 
              'http://www.dianping.com/chongqing/ch10/g110r1629', 
              'http://www.dianping.com/chongqing/ch10/g110r1632', 
              'http://www.dianping.com/chongqing/ch10/g110r6278', 
              'http://www.dianping.com/chongqing/ch10/g110r6278d500', 
              'http://www.dianping.com/chongqing/ch10/g110r1615', 
              'http://www.dianping.com/chongqing/ch10/g110r24874']
    referer=random.choice(referers)
    return referer

In [35]:
def getWebpages(url):
    
    user_agent = getUserAgent()
    referer = getReferer()
    proxies = getProxy()
    #with open('cookies.txt', 'rb') as f:
        #data = pickle.load(f)
    #for item in data:
        #cookies={item['name']:item['value']}
    headers = {
        'User-Agent': user_agent,
        'Referer': referer
        #'Cookie': str(cookies)
    }
    
    res = requests.get(url, proxies=proxies, headers=headers)
    res.encoding='utf-8'
    soup = BeautifulSoup(res.text, features='lxml')
    data = soup.find('div', class_='shop-list J_shop-list shop-all-list', id='shop-all-list')
    shops = data.find_all('li')

    return shops


In [8]:
def getData(shops):
    shopdf = pd.DataFrame(columns=['shopid', 'title', 'address', 'rating', 'recdish', 'deal'])
    for shop in shops:
        sep = '/'
        # Find shopid
        shopid = shop.find('div', class_='tit').find_all('a')[0]['data-shopid']
        # Find shop title
        title = shop.find('div', class_='tit').find_all('a')[0]['title']
        # Find shop address
        addr = shop.find('div', class_='operate J_operate Hide').find_all('a')[1]['data-address']
        # Find shop rating
        label = 'star_score score' + shop.find('div', class_='star_icon').find('span')['class'][1][4:] + ' star_score_sml'
        rating = shop.find('div', class_=label).text
        # Find recommend dishes
        recdish = []
        for item in shop.find('div', class_='recommend').find_all('a'):
            recdish.append(item.text)
        recstr = sep.join(recdish)
        # Find deal info
        deal = []
        if len(shop.find('div', class_='promo-icon J_promo_icon').find_all('a')) == 0:
            dealstr = np.nan
        else:
            for item in shop.find('div', class_='promo-icon J_promo_icon').find_all('a'):
                deal.append(item['title'])
            dealstr = sep.join(deal)

        shopdf = shopdf.append({'shopid': shopid, 'title': title, 'address': addr, 'rating': rating, 'recdish': recstr, 'deal': dealstr}, ignore_index=True)
        
    return shopdf

In [9]:
pages = 16
for page in pages:
    url = 'https://www.dianping.com/chongqing/ch10/g110r1608o3p' + str(page)
    shops = findShop(url)
    shopdf = getData(shops)
    filename = 'dianping' + str(page) + '.csv'
    shopdf.to_csv(filename)
    time.sleep(30)

TypeError: 'int' object is not iterable

In [38]:
page = 5
url = 'https://www.dianping.com/chongqing/ch10/g110r1608o3p' + str(page)
shops = getWebpages(url)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [32]:
shops

NameError: name 'shops' is not defined

In [ ]:
shopdf = getData(shops)
filename = 'dianping' + str(page) + '.csv'
shopdf.to_csv(filename)
#time.sleep(30)